# Aggie Reuse Store FQ '21: Sales Data Cleaning

Read-in entire sales data for the quarter, standardize and output a clean .csv for other analyses.

In [1]:
# Libraries
import pandas as pd

In [2]:
# Dataframe
df = pd.read_csv('../Data/popup_notclean.csv')
df.head()
df.columns = ['Date', 'Item', 'Price']

In [3]:
# View categorical attributes
print('Unique Items:', '\n') 
print(sorted(df.Item.unique()), '\n')
print('Number of Occurrences:', '\n')
print(df.Item.value_counts())

('Unique Items:', '\n')
(['UCD t-shirt', 'blouse', 'botique item', 'boutique Item', 'boutique item', 'button-down shirt', 'chemistry set', 'costume', 'dress', 'earrings', 'erasers', 'flashlight', 'glue sticks', 'hair clip', 'hand bag', 'heels', 'highlighters', 'jacket', 'jeans', 'leggings', 'long skirt', 'long sleeve shirt', 'notebook', 'notepad', 'pants', 'pencil sharpener', 'shoes', 'short skirt', 'shorts', 'slacks', 'sneakers', 'sun hat', 'sunglasses', 'sweater', 't-shirt', 'tank-top', 'vase', 'water bottle', 'windbreaker'], '\n')
('Number of Occurrences:', '\n')
boutique item        18
t-shirt              16
water bottle         14
sweater              13
long sleeve shirt    11
shorts               11
dress                11
earrings              9
button-down shirt     7
notebook              6
tank-top              5
pants                 5
blouse                5
jeans                 4
costume               3
long skirt            2
slacks                2
boutique Item      

In [4]:
data_dict = pd.read_csv('./data_dict.csv')
data_dict.tail()

,Category,Item,Clothing,Price
160,sporting_goods,U bike lock,0,10.0
161,sporting_goods,bike seat,0,10.0
162,sporting_goods,tennis rackets,0,10.0
163,miscellaneous,misc,0,NaN
164,boutique_item,boutique item,1,NaN


In [5]:
to_rep = []
for item in df.Item.unique():
    if not(item in data_dict['Item'].values) and not(item in data_dict['Category'].values):
        to_rep.append(item)
to_rep
        
    

['chemistry set',
 'botique item',
 'flashlight',
 'hair clip',
 'boutique Item',
 'costume']

In [6]:
# Create dictionary to remap 'Item' column to match Main Price List
# Create dictionary to remap 'Item' column to match Main Price List
d = {'botique item': 'boutique item',
     'boutique Item': 'boutique item',
     'chemistry set': 'school_office_supplies', # map unknown items to parent category
     'costume': 'boutique item', # costume (3 instances) not on Main Price List
     'earring':'earrings',
     'flashlight': 'household_goods', 
     'hair clip': 'accessories'}

df.replace({'Item': d}, inplace = True)

print('(Re-mapped) Unique Items:', '\n') 
print(sorted(df.Item.unique()), '\n')
print('(Re-mapped) Number of Occurrences:', '\n')
print(df.Item.value_counts())

('(Re-mapped) Unique Items:', '\n')
(['UCD t-shirt', 'accessories', 'blouse', 'boutique item', 'button-down shirt', 'dress', 'earrings', 'erasers', 'glue sticks', 'hand bag', 'heels', 'highlighters', 'household_goods', 'jacket', 'jeans', 'leggings', 'long skirt', 'long sleeve shirt', 'notebook', 'notepad', 'pants', 'pencil sharpener', 'school_office_supplies', 'shoes', 'short skirt', 'shorts', 'slacks', 'sneakers', 'sun hat', 'sunglasses', 'sweater', 't-shirt', 'tank-top', 'vase', 'water bottle', 'windbreaker'], '\n')
('(Re-mapped) Number of Occurrences:', '\n')
boutique item             24
t-shirt                   16
water bottle              14
sweater                   13
shorts                    11
long sleeve shirt         11
dress                     11
earrings                   9
button-down shirt          7
notebook                   6
tank-top                   5
pants                      5
blouse                     5
jeans                      4
hand bag                 

In [7]:
to_rep = []
for item in df.Item.unique():
    if not(item in data_dict['Item'].values) and not(item in data_dict['Category'].values):
        to_rep.append(item)
to_rep
        
    

[]

In [8]:
# Read in data dictionary and add 'Category' attribute to main data frame
category_dict = pd.Series(data_dict.Category.values, index = data_dict.Item).to_dict()

def categorize(row):
    if row['Item'] in category_dict:
        return category_dict[row['Item']]
    elif row['Item'] in category_dict.values():
        return row['Item']
    elif row['Item'] == 'boutique item':
        return 'boutique_item'
    else:
        return 'uncategorized'
    
df['Category'] = df.apply(categorize, axis = 1)

In [9]:
df.head(20)

,Date,Item,Price,Category
0,9/24/21,jeans,$5.00,pants
1,9/24/21,boutique item,$6.00,boutique_item
2,9/24/21,boutique item,$7.00,boutique_item
3,9/24/21,button-down shirt,$5.00,long_sleeves_blouses
4,9/24/21,t-shirt,$3.00,short_sleeve_tops
5,9/24/21,t-shirt,$3.00,short_sleeve_tops
6,9/24/21,pants,$5.00,pants
7,9/24/21,t-shirt,$3.00,short_sleeve_tops
8,9/24/21,boutique item,$6.00,boutique_item
9,9/24/21,dress,$6.00,skirts_shorts_dresses


In [10]:
df.loc[df['Category'] == 'uncategorized']
# hopefully this comes back empty and everything is well classified 

,Date,Item,Price,Category


In [11]:
# Add clothing y/n and suggested price from Main Price List to data frame
clothing_dict = pd.Series(data_dict.Clothing.values, index = data_dict.Category).to_dict()
clothing_dict['boutique_item'] = 1 # assuming boutique items are usually clothing (?)

def clothing(row):
    return clothing_dict[row['Category']]
    
df['Clothing'] = df.apply(clothing, axis = 1)


price_dict = pd.Series(data_dict.Price.values, index = data_dict.Item).to_dict()

def suggested_price(row):
    if row['Item'] in price_dict:
        return price_dict[row['Item']]
    else:
        return 'N/A'
    
df['Suggested Price'] = df.apply(suggested_price, axis = 1)
df.head(20)

,Date,Item,Price,Category,Clothing,Suggested Price
0,9/24/21,jeans,$5.00,pants,1,5
1,9/24/21,boutique item,$6.00,boutique_item,1,NaN
2,9/24/21,boutique item,$7.00,boutique_item,1,NaN
3,9/24/21,button-down shirt,$5.00,long_sleeves_blouses,1,5
4,9/24/21,t-shirt,$3.00,short_sleeve_tops,1,3
5,9/24/21,t-shirt,$3.00,short_sleeve_tops,1,3
6,9/24/21,pants,$5.00,pants,1,N/A
7,9/24/21,t-shirt,$3.00,short_sleeve_tops,1,3
8,9/24/21,boutique item,$6.00,boutique_item,1,NaN
9,9/24/21,dress,$6.00,skirts_shorts_dresses,1,5


In [14]:
df.to_csv('../Data/popup.csv', index = False)